In [12]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autorank import autorank, create_report, plot_stats 
import json
import glob
import re
import os

# Configuration
algorithms = ["Adam", "MAML", "Approx_GAP", "Beta_cd", "MICAT"]
datasets = ["algebra", "assist0910", "math2"]
subalgos = ["IMPACT", "NCDM"]
metrics = ["mi_acc","rmse","mae","mi_prec","mi_rec","mi_f_b","mi_auc","ma_prec","ma_rec","ma_f_b",'meta_doa','pc-er','rm']
pattern = re.compile(
    r"^CAT_launch_"
    r"(?P<dataset>.+?)_"
    r"(?P<subalgo>.+?)_"
    r"(?P<algorithm>.+?)_"
    r"\d+_all_results\.json$"
)

output_dir = "cd_plots"
os.makedirs(output_dir, exist_ok=True)

In [13]:
# Collect metric values across all valid datasets/subalgos/folds/steps
results = {m: {ds: {fold: {a: {sub : [] for a in algorithms} for sub in ['NCDM','IMPACT']} for fold in range(5)} for ds in datasets} for m in metrics}
files = glob.glob("CAT_launch_*_IMPACT_*_all_results.json") + \
        glob.glob("CAT_launch_*_NCDM_*_all_results.json")
print(f"[INFO] Found {len(files)} candidate result files.")

for file in files:
    m = pattern.match(os.path.basename(file))
    if not m:
        print(f"[WARN] Filename does not match expected pattern: {file}")
        continue
    ds, subalgo, algo = m["dataset"], m["subalgo"], m["algorithm"]
    if ds not in datasets or algo not in algorithms:
        print(f"[WARN] Skipping file due to unmatched dataset/algo: {ds}, {algo}")
        continue
    if ds=="algebra" and subalgo=="NCDM":
        continue
    try:
        with open(file, "r") as f:
            folds = json.load(f)
        for fold_idx, (pred, meta) in enumerate(folds):
            for step in pred:
                for metric in metrics:
                    value = pred[step].get(metric)
                    if value is None or np.isnan(value):
                        value = meta[step].get(metric)
                    if value is not None and not np.isnan(value):
                        value_acc[metric] += value 
                    elif metric == 'pc-er' :
                        value_acc[metric] += -2 
                    results[metric][ds][fold_idx][algo]
            for metric in metrics :
                results[metric][algo].append(value_acc[metric])
    except Exception as e:
        print(f"[ERROR] Failed to process {file}: {e}")

[INFO] Found 28 candidate result files.


In [ ]:
import numpy as np
nb_methods = len(results.keys())
A = np.zeros((nb_methods,nb_methods))
    

for i_folds in range(5) : 
    for method in results.keys(): 
        A

In [9]:
results['MICAT'][0][0]['1']['mi_acc']

{'mi_acc': 0.7809749245643616,
 'rmse': 0.46800121665000916,
 'mae': 0.2190251350402832,
 'mi_prec': 0.93740314245224,
 'mi_rec': 0.7871454358100891,
 'mi_f_b': 0.8557283878326416,
 'mi_auc': 0.7694941055596793,
 'ma_prec': 0.6826875805854797,
 'ma_rec': 0.7694940567016602,
 'ma_f_b': 0.7005914449691772}

In [12]:
import numpy as np
r1 = {m : {step:[] for step in range(16)} for m in results.keys()}
std = {m : {step:0 for step in range(16)} for m in results.keys()}
for method in results.keys():
    for i_fold, fold in enumerate(results[method]) : 
        for step, values in fold[0].items() :
            r1[method][int(step)].append(values['mi_acc'])
for method in results.keys():
    for step, values in fold[0].items() :
            std[method][int(step)] = (np.array(r1[method][int(step)])).std()
    

In [13]:
print(std)

{'MICAT_INIT': {0: np.float64(0.015311678450248684), 1: np.float64(0.015576789025920842), 2: np.float64(0.015385880968059398), 3: np.float64(0.015385880968059398), 4: np.float64(0.01495398426626826), 5: np.float64(0.014447691130185122), 6: np.float64(0.014166756134757501), 7: np.float64(0.01413022158120815), 8: np.float64(0.013919491153503964), 9: np.float64(0.013765917490149899), 10: np.float64(0.014344688140669363), 11: np.float64(0.014208522056182345), 12: np.float64(0.014023492916829298), 13: np.float64(0.013899481730962938), 14: np.float64(0.013429734904105307), 15: np.float64(0.012630633313053757)}, 'MICAT_INIT_REG': {0: np.float64(0.015311678450248684), 1: np.float64(0.015576789025920842), 2: np.float64(0.015385880968059398), 3: np.float64(0.015385880968059398), 4: np.float64(0.01495398426626826), 5: np.float64(0.014447691130185122), 6: np.float64(0.014166756134757501), 7: np.float64(0.01413022158120815), 8: np.float64(0.013919491153503964), 9: np.float64(0.013765917490149899), 

In [4]:
r2 = {m : {step:0 for step in range(16)} for m in results.keys()}
for method in results.keys():
    for i_fold, fold in enumerate(results[method]) : 
        for step, values in fold[0].items() :
            r2[method][int(step)] += (values['mi_acc']/results['MICAT'][i_fold][0][step]['mi_acc'])*100/5
    

In [5]:
r2

{'MICAT_INIT': {0: 99.02847874213727,
  1: 98.84867825206221,
  2: 98.39380254768244,
  3: 98.00012132891382,
  4: 97.60249170416606,
  5: 97.39125053835295,
  6: 96.96062956547573,
  7: 96.64777311696423,
  8: 96.06787518520221,
  9: 95.11942855436175,
  10: 95.11093716472195,
  11: 94.71310027009267,
  12: 94.0451955942788,
  13: 93.51124520596912,
  14: 93.41915007648996,
  15: 93.32953257486933},
 'MICAT_INIT_REG': {0: 99.02847874213727,
  1: 98.84867825206221,
  2: 98.39380254768244,
  3: 98.00012132891382,
  4: 97.60249170416606,
  5: 97.39125053835295,
  6: 96.96062956547573,
  7: 96.64777311696423,
  8: 96.06787518520221,
  9: 95.11942855436175,
  10: 95.11093716472195,
  11: 94.71310027009267,
  12: 94.0451955942788,
  13: 93.51124520596912,
  14: 93.41915007648996,
  15: 93.32953257486933},
 'MICAT': {0: 100.0,
  1: 100.0,
  2: 100.0,
  3: 100.0,
  4: 100.0,
  5: 100.0,
  6: 100.0,
  7: 100.0,
  8: 100.0,
  9: 100.0,
  10: 100.0,
  11: 100.0,
  12: 100.0,
  13: 100.0,
  14: 1